In [47]:
from PriorityQueue import PriorityQueue
from Cell import Position, Cell


from typing import Iterable, Optional

def path_traceback(start_state: Cell, goal_state: Cell) -> list[Cell]:
    path = []
    while goal_state != start_state:
        path.append(goal_state)
        goal_state = goal_state.parent
    path.append(start_state)
    return list(reversed(path))

def astar_vacuum(grid_dim: tuple[int, int],
                 dirty_cells: Iterable[Position],
                 start: Position, *,
                 do_traceback: bool = False)\
                -> tuple[Cell, Optional[list[Cell]]]:
    my_queue: PriorityQueue[Cell] = PriorityQueue()
    visited = {}
    goals = []

    start_node = Cell(position=start, action='move', status='clean',
                      grid_dim=grid_dim, dirty_cells=dirty_cells)
    start_node.calc_cost()
    my_queue.push(start_node)
    while my_queue:
        cur = my_queue.pop()

        if len(cur.dirty_cells) == 0:
            goals.append(cur)
            visited[cur] = cur
            continue
        
        visited[cur] = cur
        for neighbour in cur.expand_cell():
            neighbour.calc_cost()
            if neighbour in visited:
                if visited[cur].cost < neighbour.cost: continue
            my_queue.push(neighbour)

    # cur = min(goals, key=lambda x: x.cost)
    # traceback = path_traceback(start_node, cur) if do_traceback else None
    # return cur, traceback
    return goals

In [9]:
dirty = [Position(1, 1),
         Position(1, 6),
         Position(3, 8),
         Position(5, 8),
         Position(8, 6)]
start = Position(4, 5)
grid_dim = (8, 8)

In [48]:
goal = astar_vacuum(grid_dim, dirty, start, do_traceback=True)

In [54]:
goal[4].cost

105

In [ ]:
start_node = Cell(start, 'move', 'clean', grid_dim, dirty)
start_node.calc_cost()
for cell in path_traceback(start_node, goal[1]):
    print(cell.action, cell.position, cell.cost)

In [55]:
start_node = Cell(start, 'move', 'clean', grid_dim, dirty)
start_node.calc_cost()
next_cell = start_node.expand_cell()

In [ ]:
next_cell[2]

(5, 5)